In [5]:
import os
import json
import numpy as np
import re
from collections import defaultdict
from pprint import pprint

In [2]:
np.random.seed(0)

In [204]:
dir = '../../logs/compute_pp_archive_train'
os.getcwd()

'/home/laetitia/work/OpenELM/analysis_P3/quality'

In [205]:
puzzles_train = json.load(open(os.path.join(dir, 'puzzles_train_compute_pp.json'), 'r'))
puzzles_test = json.load(open(os.path.join(dir, 'puzzles_test_compute_pp.json'), 'r'))
puzzles_ELM_NLP = json.load(open(os.path.join(dir, 'puzzles_ELM_NLP_0_compute_pp.json'), 'r'))
puzzles_ACES_smart = json.load(open(os.path.join(dir, 'puzzles_ACES_smart_0_compute_pp.json'), 'r'))
puzzles_rd_gen = json.load(open(os.path.join(dir, 'puzzles_rd_gen_0_compute_pp.json'), 'r'))

In [5]:
# sample 20 puzzles for each of the generated sets
for set, setname in zip([puzzles_ELM_NLP, puzzles_rd_gen, puzzles_ACES_smart], ['elm', 'rd_gen', 'aces']):
    sampled_idx = np.random.choice(len(set), 20, replace=False)
    for idx in sampled_idx:
        print(f'Puzzle {idx} from {setname}')
        print(set[idx]['description'])
        print(set[idx]['program_str'])
        print('')
        print('-' * 20)
    print('')
    print('=' * 80)

Puzzle 1247 from elm
['The puzzle checks if a given string is a palindrome or not.']

def f(s: str) -> bool:
    return s == s[::-1]

def g() -> str:
    return "radar"

assert f(g()) == True


--------------------
Puzzle 609 from elm
['The puzzle requires defining a function f(lst) that checks if a list contains unique elements, and a function g(size, start, step) that generates a list of numbers based on the input parameters. The puzzle tests whether the generated list by g() satisfies the uniqueness condition in f().']

def f(lst) -> bool:
    return len(lst) == len(set(lst))

def g(size=5, start=1, step=2):
    return list(range(start, start + size * step, step))

assert f(g()) == True


--------------------
Puzzle 393 from elm
['The puzzle requires defining a function f(nums: list) that checks if a list of numbers contains both even and odd numbers. The solution function g() generates a list of 5 random numbers between 1 and 100 and returns it. The assertion checks if f(g()) retur

In [6]:
set[0]['description']

[["The puzzle requires finding a number that, when squared, starts with the digits '123456789'."]]

In [7]:
fpattern = 'def f\(.*\).*:'
gpattern = 'def g\(.*\).*:'

def format_generated(p):
    if isinstance(p["description"][0], list):
        descr = f'\n    """\n    {p["description"][0][0]}\n    """'
    else:
        descr = f'\n    """\n    {p["description"][0]}\n    """'
    pstring = p['program_str']
    match_f_name = re.findall(fpattern, pstring)[0]
    # remove typing imports etc
    pstring = match_f_name + ''.join(pstring.split(match_f_name)[1:])
    # add description string
    pstring = pstring.replace(match_f_name, match_f_name + descr)

    gmatch = re.findall(gpattern, pstring)[0]
    pstring = pstring.replace(gmatch, '\n' + gmatch)
    return pstring, descr

In [8]:
isinstance('lele', list)

False

In [9]:
print(format_generated(set[-1])[0])

def f(solution, s='radar') -> bool:
    """
    Given a function f that checks if a string is a palindrome and a function g that creates a palindrome from a given string, the puzzle requires finding a string that is a palindrome.
    """
    return solution == solution[::-1]


def g(s='radar'):
    return s + s[-2::-1]

assert f(g()) == True



In [84]:
def format_p3(p):
    descr = p['sol_docstring']
    fpstring = p['sat'].replace('def sat(', 'def f(')
    match_f_name = re.findall(fpattern, fpstring)[0]
    # remove typing imports etc
    fpstring = match_f_name + ''.join(fpstring.split(match_f_name)[1:])
    # add description string
    fpstring = fpstring.replace(match_f_name, '\n'.join([match_f_name, descr]))
    gpstring = '\n'.join([p['sol_header'].replace('def sol(', 'def g('), np.random.choice(p['sol_bodies'])])  # random solution
    return fpstring + '\n\n' + gpstring, descr

In [21]:
print(format_p3(puzzles_train[0])[0])

def f(s: str):
    """Find a string with 1000 'o's but no two adjacent 'o's."""
    return s.count('o') == 1000 and s.count('oo') == 0
def g():
    return ('h' + 'o') * 1000


In [41]:
selected_gen_dict = {
    'elm': [1241, 190, 639, 1290, 929, 646],
    'rd_gen': [15, 1099, 1233, 55, 344, 746, 53],
    'aces': [237, 162, 822, 365, 903, 948, 918],
}

selected_p3_name_list = ['SeparateParenGroups', 'FindContainers', 'LargestPrimeFactor', 'HeronTriangle', 'WeirdDecodeVowels', 'ExponentialCoinMoves', 'FindRepeats', 'LineIntersection', 'IfProblemWithOr', 'ZipStr', 'SumOfDigits', 'Study_5', 'Study_17', 'Tutorial2', 'Tutorial3', 'Lehmer', 'GCD_multi', 'Abbreviate', 'ListIndex', 'ShortestPath']

In [42]:
formatted_p3 = []

selected_defaultdict = defaultdict(list)

for i, p in enumerate(puzzles_train + puzzles_test):
    if i < len(puzzles_train):
        origin = 'train'
    else:
        origin = 'test'
    name_cleaned = p['name'].split(':')[0]
    p['origin'] = origin
    if name_cleaned in selected_p3_name_list:
        selected_defaultdict[name_cleaned].append(p)

In [43]:
pprint(selected_defaultdict)

defaultdict(<class 'list'>,
            {'Abbreviate': [{'ans_type': 'str',
                             'final_losses': [0.2708492875099182,
                                              0.6547725796699524,
                                              0.21053525805473328,
                                              2.663637399673462,
                                              1.3769851922988892,
                                              0.6277777552604675,
                                              0.9691902995109558,
                                              2.37506365776062,
                                              1.4403624534606934,
                                              0.6576752066612244,
                                              0.33830776810646057,
                                              0.7193517088890076,
                                              0.8574972152709961,
                                              1.4177356958389282,
 

In [44]:
len(selected_defaultdict['Abbreviate'])

5

In [48]:
selected_p3 = []

for name, vals in selected_defaultdict.items():
    puz = np.random.choice(vals)
    formatted_puz, descr = format_p3(puz)
    puz['formatted'] = formatted_puz
    puz['description'] = descr
    selected_p3.append(puz)

In [61]:
len(selected_p3)

20

In [62]:
print(selected_p3[4]['description'])

    """
    Find the intersection of two lines.
    Solution should be a list of the (x,y) coordinates.
    Accuracy of fifth decimal digit is required.
    """


In [67]:
selected_gen = []
for idx in selected_gen_dict['elm']:
    puz = puzzles_ELM_NLP[idx]
    formatted_puz, descr = format_generated(puz)
    puz['formatted'] = formatted_puz
    puz['description'] = descr
    puz['origin'] = 'elm'

    selected_gen.append(puz)
for idx in selected_gen_dict['rd_gen']:
    puz = puzzles_rd_gen[idx]
    formatted_puz, descr = format_generated(puz)
    puz['formatted'] = formatted_puz
    puz['description'] = descr
    puz['origin'] = 'rd_gen'

    selected_gen.append(puz)
for idx in selected_gen_dict['aces']:
    puz = puzzles_ACES_smart[idx]
    formatted_puz, descr = format_generated(puz)
    puz['formatted'] = formatted_puz
    puz['description'] = descr
    puz['origin'] = 'aces'

    selected_gen.append(puz)

In [68]:
len(selected_gen)

20

In [70]:
print(selected_gen[19]['description'])


    """
    The puzzle requires defining a function f that takes a list of moves as input and checks if the moves are valid to stay within a board of given size. The solution function g generates a list of moves to navigate within an 8x8 board.
    """


In [ ]:
# tag puzzles with remarks

In [252]:
i = 0

In [246]:
print(i)
print(selected_p3[i]['formatted'])

19
def f(item: int, li=[93, -13, -56, 19], index=2):
    """Find the item whose first index in li is index"""
    return li.index(item) == index
def g(li=[93, -13, -56, 19], index=2):
    return li[index]


In [247]:
notes = ''''''
selected_p3[i]['notes'] = notes
i += 1

In [204]:
for i in range(len(selected_p3)):
    selected_gen[i]['notes'] = selected_p3[i]['notes']

In [291]:
print(i)
print(selected_gen[i]['formatted'])

19
def f(moves: list) -> bool:
    """
    The puzzle requires defining a function f that takes a list of moves as input and checks if the moves are valid to stay within a board of given size. The solution function g generates a list of moves to navigate within an 8x8 board.
    """
    def is_valid_move(move):
        return move[0] == 0 or move[1] == 0

    current_position = (0, 0)
    for move in moves:
        new_position = (current_position[0] + move[0], current_position[1] + move[1])
        if not is_valid_move(move) or new_position[0] < 0 or new_position[1] < 0:
            return False
        current_position = new_position

    return True


def g(board_size=8):
    moves = [(0, 1), (1, 0), (0, -1), (-1, 0)]
    return moves

assert f(g()) == True



In [292]:
notes = 'Underconstrained puzzle, checks if generated moves are valid, but almost all are.'
selected_gen[i]['notes'] = notes
i += 1

In [172]:
os.getcwd()

'/home/laetitia/work/OpenELM/analysis_P3/quality'

In [173]:
os.chdir('/home/laetitia/work/OpenELM/survey')

In [250]:
print(selected_gen[3]['notes'])

Simple addition puzzle


In [295]:
selected_p3[1]['notes']

'Pretty simple, no algorithmic answer.'

In [296]:
selected_gen[1]['notes']

'Simple puzzle. Solution a bit too complicated.'

In [293]:
with open('puzzle_survey_v1.json', 'w') as f:
    json.dump(selected_p3 + selected_gen, f)

In [251]:
with open('puzzle_survey_v1_p3.json', 'w') as f:
    json.dump(selected_p3, f)

In [4]:
with open('/home/laetitia/work/OpenELM/survey/puzzle_survey_v1.json', 'r') as f:
    res = json.load(f)

In [117]:
len(res)

40

In [118]:
print(res[23]['formatted'])

def f(text: str, target='a') -> bool:
    """
    The puzzle involves two functions, f and g, where f checks if a target character is present in a given text, and g returns the alphabet if the target character is 'a'. The assertion checks if f(g()) returns True.
    """
    return target in text


def g(target='a'):
    return 'abcdefghijklmnopqrstuvwxyz' if target in 'abcdefghijklmnopqrstuvwxyz' else ''

assert f(g()) == True



In [119]:
# clean the assert
pattern = 'assert f\(g\(.*'
def clean_assert(program_str, origin):
    # add some formatting for p3 puzzles
    if origin in ['train', 'test']:
        program_str = program_str.replace('def g(', '\ndef g(')
    
    program_str = program_str.replace('\n\n\n', '\n\n')

    findall = re.findall(pattern, program_str)
    if findall:
        return program_str.split(findall[0])[0].strip()
    else:
        return program_str

In [120]:
for p in res:
    p['formatted'] = clean_assert(p['formatted'], p['origin'])

In [122]:
print(res[23]['formatted'])

def f(text: str, target='a') -> bool:
    """
    The puzzle involves two functions, f and g, where f checks if a target character is present in a given text, and g returns the alphabet if the target character is 'a'. The assertion checks if f(g()) returns True.
    """
    return target in text

def g(target='a'):
    return 'abcdefghijklmnopqrstuvwxyz' if target in 'abcdefghijklmnopqrstuvwxyz' else ''


In [123]:
print(p['formatted'])

def f(moves: list) -> bool:
    """
    The puzzle requires defining a function f that takes a list of moves as input and checks if the moves are valid to stay within a board of given size. The solution function g generates a list of moves to navigate within an 8x8 board.
    """
    def is_valid_move(move):
        return move[0] == 0 or move[1] == 0

    current_position = (0, 0)
    for move in moves:
        new_position = (current_position[0] + move[0], current_position[1] + move[1])
        if not is_valid_move(move) or new_position[0] < 0 or new_position[1] < 0:
            return False
        current_position = new_position

    return True

def g(board_size=8):
    moves = [(0, 1), (1, 0), (0, -1), (-1, 0)]
    return moves


In [6]:
samples_per_origin = defaultdict(list)

In [7]:
for p in res:
    samples_per_origin[p['origin']].append(p)

In [8]:
num_samples_per_origin = {(k, len(v)) for k, v in samples_per_origin.items()}

In [9]:
num_samples_per_origin = dict(
    aces=3,
    elm=2,
    rd_gen=2,
    test=3,
    train=5,
)

In [10]:
np.random.seed(43)

In [11]:
newset = []
notset = []
for name, values in samples_per_origin.items():
    sampled_idx = np.random.choice(len(values), num_samples_per_origin[name], replace=False)
    newset += [values[i] for i in sampled_idx]
    notset += [values[i] for i in range(len(values)) if i not in sampled_idx]

In [12]:
len(newset)
len(notset)

25

In [141]:
for i, p in enumerate(newset):
    print(p['formatted'])
    print('\n=====\n')

def f(s: str, substrings=['cute', 'rysucajaxuno']):
    """
    Find a string that contains each string in substrings alternating, e.g., 'cdaotg' for 'cat' and 'dog'
    """
    return all(sub in s[i::len(substrings)] for i, sub in enumerate(substrings))

def g(substrings=['cute', 'rysucajaxuno']):
    m = max(len(s) for s in substrings)
    return "".join([(s[i] if i < len(s) else " ") for i in range(m) for s in substrings])

=====

def f(indices: List[int], a0=2827347):
    """
    Find a repeating integer in an infinite sequence of integers, specifically the indices for which the same value
    occurs 1000 times. The sequence is defined by a starting value a_0 and each subsequent term is:
    a_{n+1} = the square root of a_n if the a_n is a perfect square, and a_n + 3 otherwise.

    For a given a_0 (that is a multiple of 3), the goal is to find 1000 indices where the a_i's are all equal.

    Sample input:
    9

    Sample output:
    [0, 3, 6, ..., 2997]

    The sequence startin

In [140]:
for i, p in enumerate(notset):
    print(p['formatted'])
    print('\n=====\n')

def f(li: List[int]):
    """Find a list integers such that the integer i occurs i times, for i = 0, 1, 2, ..., 9."""
    return all([li.count(i) == i for i in range(10)])

def g():
    return [i for i in range(10) for j in range(i)]

=====

def f(i: int):
    """Find a number whose decimal representation is *a longer string* when you add 1,000 to it than when you add 1,001."""
    return len(str(i + 1000)) > len(str(i + 1001))

def g():
    return -1001

=====

def f(x: str, s=679):
    """Find a number that its digits sum to a specific value."""
    return s == sum([int(d) for d in x])

def g(s=679):
    return int(s / 9) * '9' + str(s % 9)

=====

def f(e: List[int], a=-18517001, b=-13662763, c=-11156613, d=9271005):
    """
    Find the intersection of two lines.
    Solution should be a list of the (x,y) coordinates.
    Accuracy of fifth decimal digit is required.
    """
    x = e[0] / e[1]
    return abs(a * x + b - c * x - d) < 10 ** -5

def g(a=-18517001, b=-13662763, c=-1115

In [13]:
lehmer = [p for p in notset if 'name' in p and 'lehmer' in p['name'].lower()][0]

In [14]:
lehmer

{'name': 'Lehmer:0',
 'sat': 'def sat(n: int):\n    return pow(2, n, n) == 3',
 'ans_type': 'int',
 'sol_header': 'def sol():',
 'sol_docstring': '    """Find n  such that 2^n mod n = 3"""',
 'sol_bodies': ['    return 4700063497'],
 'module': 'number_theory.py',
 'notes': 'Hard number theory puzzle, no algorithmic answer. \n\nAccording to [The Strong Law of Large Numbers](https://doi.org/10.2307/2322249) Richard K. Guy states that\nD. H. & Emma Lehmer discovered that 2^n = 3 (mod n) for n = 4700063497,\nbut for no smaller n > 1.',
 'weight': 1.0,
 'origin': 'train',
 'fitnessPP': -0.318130224943161,
 'final_losses': [2.9315078258514404,
  0.4021201431751251,
  2.7738747596740723,
  2.6642990112304688,
  1.3305975198745728,
  2.787921905517578,
  0.6401402354240417,
  1.555601716041565,
  2.362389326095581,
  0.806269645690918,
  0.2546146512031555,
  0.943902850151062,
  0.7497960329055786,
  1.1959009170532227,
  1.059122085571289,
  2.850597381591797,
  2.8500750064849854,
  0.66186

In [146]:
[p['name'].lower() for p in notset if 'name' in p]

['study_5:0',
 'study_17:0',
 'sumofdigits:0',
 'lineintersection:4',
 'ifproblemwithor:2',
 'shortestpath:3',
 'lehmer:0',
 'tutorial2:0',
 'separateparengroups:1',
 'largestprimefactor:3',
 'herontriangle:1',
 'exponentialcoinmoves:2']

In [147]:
'lehmer' in 'lehmer:0'

True

In [15]:
import json
with open('/home/laetitia/work/OpenELM/survey/puzzle_survey_v2.json', 'r') as f:
    newset = json.load(f)

In [16]:
# newset.pop(0)
newset = [lehmer] + newset

In [17]:
with open('/home/laetitia/work/OpenELM/survey/puzzle_survey_v2.json', 'w') as f:
    json.dump(newset, f)

In [18]:
len(newset)

15

In [181]:
pattern = '\n    """.*"""\n'
def remove_docstring(program_str):
    docstring = re.findall(pattern, program_str, re.DOTALL)[0]  # all puzzles have a docstring
    program_str = program_str.replace(docstring, '\n')
    return program_str

In [182]:
print(remove_docstring(newset[0]['formatted']))

def f(indices: List[int], a0=2827347):
    assert a0 >= 0 and a0 % 3 == 0, "Hint: a_0 is a multiple of 3."
    s = [a0]
    for i in range(max(indices)):
        s.append(int(s[-1] ** 0.5) if int(s[-1] ** 0.5) ** 2 == s[-1] else s[-1] + 3)
    return len(indices) == len(set(indices)) == 1000 and min(indices) >= 0 and len({s[i] for i in indices}) == 1

def g(a0=2827347):
    n = a0
    ans = []
    i = 0
    while len(ans) < 1000:
        if n == 3:  # use the fact that 3 will repeat infinitely often
            ans.append(i)
        n = int(n ** 0.5) if int(n ** 0.5) ** 2 == n else n + 3
        i += 1
    return ans


In [170]:
lele = '\n    """lelele"""\n'
re.findall(pattern, lele)

[]

In [178]:
re.findall(pattern, newset[0]['formatted'], re.DOTALL)

['\n    """\n    Find a repeating integer in an infinite sequence of integers, specifically the indices for which the same value\n    occurs 1000 times. The sequence is defined by a starting value a_0 and each subsequent term is:\n    a_{n+1} = the square root of a_n if the a_n is a perfect square, and a_n + 3 otherwise.\n\n    For a given a_0 (that is a multiple of 3), the goal is to find 1000 indices where the a_i\'s are all equal.\n\n    Sample input:\n    9\n\n    Sample output:\n    [0, 3, 6, ..., 2997]\n\n    The sequence starting with a0=9 is [9, 3, 6, 9, 3, 6, 9, ...] thus a_n at where n is a multiple of 3 are\n    all equal in this case.\n    """\n']

In [169]:
newset[0]['formatted']

'def f(indices: List[int], a0=2827347):\n    """\n    Find a repeating integer in an infinite sequence of integers, specifically the indices for which the same value\n    occurs 1000 times. The sequence is defined by a starting value a_0 and each subsequent term is:\n    a_{n+1} = the square root of a_n if the a_n is a perfect square, and a_n + 3 otherwise.\n\n    For a given a_0 (that is a multiple of 3), the goal is to find 1000 indices where the a_i\'s are all equal.\n\n    Sample input:\n    9\n\n    Sample output:\n    [0, 3, 6, ..., 2997]\n\n    The sequence starting with a0=9 is [9, 3, 6, 9, 3, 6, 9, ...] thus a_n at where n is a multiple of 3 are\n    all equal in this case.\n    """\n    assert a0 >= 0 and a0 % 3 == 0, "Hint: a_0 is a multiple of 3."\n    s = [a0]\n    for i in range(max(indices)):\n        s.append(int(s[-1] ** 0.5) if int(s[-1] ** 0.5) ** 2 == s[-1] else s[-1] + 3)\n    return len(indices) == len(set(indices)) == 1000 and min(indices) >= 0 and len({s[i] for 

In [183]:
for p in newset:
    p['formatted_nodoc'] = remove_docstring(p['formatted'])

In [185]:
len(res)

40

In [186]:
with open('/home/laetitia/work/OpenELM/survey/puzzle_survey_v1.json', 'w') as f:
    json.dump(res, f)

In [223]:
len(puzzles_train) + len(puzzles_rd_gen) + len(puzzles_ACES_smart) + len(puzzles_ELM_NLP)

4138

In [224]:
len(puzzles_rd_gen) + len(puzzles_ACES_smart) + len(puzzles_ELM_NLP)

3562

In [190]:
# name generated puzzles

In [196]:
os.getcwd()

'/home/laetitia/work/OpenELM/analysis_P3/quality'

In [214]:
dir = '/home/laetitia/work/OpenELM/logs/latest'
o_puzzles_ELM_NLP = json.load(open(os.path.join(dir, 'puzzles_ELM_NLP_0.json'), 'r'))
o_puzzles_ACES_smart = json.load(open(os.path.join(dir, 'puzzles_ACES_smart_0.json'), 'r'))
o_puzzles_rd_gen = json.load(open(os.path.join(dir, 'puzzles_rd_gen_0.json'), 'r'))

In [213]:
dir = '/home/laetitia/work/OpenELM/logs/compute_pp_archive_train'
puzzles_ELM_NLP = json.load(open(os.path.join(dir, 'puzzles_ELM_NLP_0_compute_pp.json'), 'r'))
puzzles_ACES_smart = json.load(open(os.path.join(dir, 'puzzles_ACES_smart_0_compute_pp.json'), 'r'))
puzzles_rd_gen = json.load(open(os.path.join(dir, 'puzzles_rd_gen_0_compute_pp.json'), 'r'))

In [220]:
for i, p in enumerate(o_puzzles_rd_gen):
    p['name'] = f'RdGen_{i}:0'
for i, p in enumerate(o_puzzles_ELM_NLP):
    p['name'] = f'ELMMLP_{i}:0'
for i, p in enumerate(o_puzzles_ACES_smart):
    p['name'] = f'ACES_{i}:0'

In [201]:
puzzles_ACES_smart[-1]['name']

'ACES_960:0'

In [222]:
dir = '/home/laetitia/work/OpenELM/logs/latest'
json.dump(o_puzzles_ELM_NLP, open(os.path.join(dir, 'puzzles_ELM_NLP_0.json'), 'w'))
json.dump(o_puzzles_ACES_smart, open(os.path.join(dir, 'puzzles_ACES_smart_0.json'), 'w'))
json.dump(o_puzzles_rd_gen, open(os.path.join(dir, 'puzzles_rd_gen_0.json'), 'w'))

In [206]:
len(puzzles_ACES_smart)

961

In [208]:
puzzles_ACES_smart[0].keys()

dict_keys(['fitness', 'program_str', 'emb', 'explanation_emb', 'idx_generation', 'target_skills', 'puzzle_history', 'problem_func', 'solution_func', 'quality', 'description', 'is_valid', 'is_valid_explanation', 'original_losses', 'fitnessPP', 'final_losses', 'name'])

In [212]:
len([p for p in puzzles_ACES_smart if 'fitnessPP' in p])

389

In [215]:
len(puzzles_ACES_smart) == len(o_puzzles_ACES_smart)

True

In [216]:
len(puzzles_ELM_NLP) == len(o_puzzles_ELM_NLP)

True

In [217]:
len(puzzles_rd_gen) == len(o_puzzles_rd_gen)

True

In [218]:
puzzles_rd_gen[0]['name']

'RdGen_0:0'

In [219]:
o_puzzles_rd_gen[0]['name']

KeyError: 'name'

In [221]:
for p, pp in zip(o_puzzles_rd_gen, puzzles_rd_gen):
    assert p['name'] == pp['name']